## LAB1_DEEPLEARNING
- Find datasets here - [link](https://drive.google.com/drive/folders/1FvSShuY_8QxPmsrHvuSkFYYc5ZBNbetU?usp=sharing)
- download the dataset and change the path n run 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage

%matplotlib inline

In [2]:
import h5py

def inspect_h5file(file_path):
    with h5py.File(file_path, "r") as file:
        def printname(name, obj):
            if isinstance(obj, h5py.Dataset):
                print(f"Dataset: {name}")
            elif isinstance(obj, h5py.Group):
                print(f"Group: {name}")

        file.visititems(printname)

inspect_h5file('/content/drive/MyDrive/data-set-sem5/lab1/Train.h5')
inspect_h5file('/content/drive/MyDrive/data-set-sem5/lab1/Test.h5')


Dataset: images
Dataset: labels
Dataset: images
Dataset: labels


In [4]:
def load_dataset():
    train_dataset = h5py.File('/content/drive/MyDrive/data-set-sem5/lab1/Train.h5', "r")
    train_set_x_orig = np.array(train_dataset["images"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["labels"][:]) # your train set labels

    test_dataset = h5py.File('/content/drive/MyDrive/data-set-sem5/lab1/Test.h5', "r")
    test_set_x_orig = np.array(test_dataset["images"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["labels"][:]) # your test set labels

    # classes = np.array(test_dataset["list_classes"][:]) # the list of classes

    # train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    # test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig

In [5]:
def initialize_parameters(dim, num_classes):
    w = np.zeros((dim, num_classes))
    b = np.zeros((num_classes))
    return w, b

In [6]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
    return exp_z / np.sum(exp_z, axis=0, keepdims=True)

In [7]:
def compute_cost_grads(w, b, X, Y):
    m = X.shape[1]
    z = np.dot(w.T, X) + b[:, np.newaxis]
    A = softmax(z)

    cost = -np.sum(Y * np.log(A + 1e-8)) / m

    dz = A - Y
    dw = np.dot(X, dz.T) / m
    db = np.sum(dz, axis=1) / m

    grads = {"dw": dw, "db": db}

    return grads, cost

In [8]:
def one_hot_encode(labels, num_classes):
    return np.eye(num_classes)[labels.reshape(-1)].T

In [23]:
def gradient_descent(X, Y, num_classes, learning_rate, num_iterations):
    dim = X.shape[0]
    w, b = initialize_parameters(dim, num_classes)

    for i in range(num_iterations):
        grads, cost = compute_cost_grads(w, b, X, Y)

        w -= learning_rate * grads['dw']
        b -= learning_rate * grads['db']

        # Implement learning rate decay
        learning_rate *= 0.99

        if i % 100 == 0:
            print(f"Iteration {i}: Cost {cost}")

    return w, b


In [24]:
def predict(w, b, X):
    z = np.dot(w.T, X) + b[:, np.newaxis]
    A = softmax(z)
    return np.argmax(A, axis=0)


In [25]:
# Implement the multilayer model function
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously.

    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (one-hot vector) of shape (number of classes, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (one-hot vector) of shape (number of classes, m_test)
    num_iterations -- number of iterations to optimize the parameters
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- if set to True, this will print the cost every 100 iterations

    Returns:
    d -- dictionary containing information about the model.
    """

    # Initialize parameters with zeros
    dim, num_classes = X_train.shape[0], Y_train.shape[0]
    w, b = initialize_parameters(dim, num_classes)

    # Gradient descent optimization
    parameters = gradient_descent(X_train, Y_train, num_classes, learning_rate, num_iterations)

    w, b = parameters

    # Predict test/train set examples
    Y_prediction_train = predict(w, b, X_train)
    Y_prediction_test = predict(w, b, X_test)

    # Print train/test Errors
    train_accuracy = np.mean(Y_prediction_train == np.argmax(Y_train, axis=0))
    test_accuracy = np.mean(Y_prediction_test == np.argmax(Y_test, axis=0))

    print(f"Train Accuracy: {train_accuracy * 100}%")
    print(f"Test Accuracy: {test_accuracy * 100}%")

    d = {"train_accuracy": train_accuracy,
         "test_accuracy": test_accuracy,
         "w": w,
         "b": b,
         "learning_rate": learning_rate,
         "num_iterations": num_iterations}

    return d

In [26]:
train_images, train_labels, test_images, test_labels = load_dataset()

In [27]:
train_images_flat = train_images.reshape(train_images.shape[0], -1).T / 255.0
test_images_flat = test_images.reshape(test_images.shape[0], -1).T / 255.0

In [28]:
num_classes = 5
train_labels_one_hot = one_hot_encode(train_labels, num_classes)
test_labels_one_hot = one_hot_encode(test_labels, num_classes)

In [30]:
d = model(train_images_flat, train_labels_one_hot, test_images_flat, test_labels_one_hot, num_iterations=3000, learning_rate=0.006, print_cost=True)

Iteration 0: Cost 1.6094378624341021
Iteration 100: Cost 2.0608597757910463
Iteration 200: Cost 1.0741010088695873
Iteration 300: Cost 1.0576980935565206
Iteration 400: Cost 1.0523962696329634
Iteration 500: Cost 1.0505254441199157
Iteration 600: Cost 1.0498492118618397
Iteration 700: Cost 1.0496027994957164
Iteration 800: Cost 1.0495127517340392
Iteration 900: Cost 1.049479810974048
Iteration 1000: Cost 1.049467756217077
Iteration 1100: Cost 1.0494633441378707
Iteration 1200: Cost 1.0494617292213013
Iteration 1300: Cost 1.0494611381159205
Iteration 1400: Cost 1.0494609217530795
Iteration 1500: Cost 1.0494608425573957
Iteration 1600: Cost 1.0494608135692292
Iteration 1700: Cost 1.0494608029586248
Iteration 1800: Cost 1.0494607990748008
Iteration 1900: Cost 1.0494607976531953
Iteration 2000: Cost 1.0494607971328418
Iteration 2100: Cost 1.0494607969423757
Iteration 2200: Cost 1.049460796872659
Iteration 2300: Cost 1.0494607968471406
Iteration 2400: Cost 1.0494607968377998
Iteration 2500: